#  Generate Some Transfered Sentences and get Style Scores

## Imports

In [18]:
%load_ext autoreload
%autoreload 2
    
import sys, os
import numpy as np
sys.path.append('../paraphrase/')
sys.path.append('../jointclassifier/')
from paraphraser_args import ModelArguments as pma, DataTrainingArguments as pda, TrainingArguments as pta
from paraphraser_dataloader import load_dataset as pld, load_dataset_style as lds
from paraphraser_dataloader import load_dataset_pseudo as ldp
from paraphraser_trainer import ParaphraserTrainer
from transformers import AutoConfig, AutoTokenizer, AutoModelWithLMHead, HfArgumentParser
from joint_args import ModelArguments as jma, DataTrainingArguments as jda, TrainingArguments as jta
from joint_dataloader import load_dataset as jld
from joint_trainer import JointTrainer
from joint_model_v1 import JointSeqClassifier

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from tqdm.notebook import tqdm, trange
from torch import cuda, no_grad

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load in desired dataset and paraphraser model
In the cell below, define the dataset you want to work with and the paraphraser model (here a `"t5-small"` [from Hugging Face](https://huggingface.co/t5-small))

In [19]:
data_dir = '../data/pseudo/'
#"../data/processed_filtered/"

In [20]:
mode = 'dev'
paraphrase_model_name = "t5_transfer_shakespeare"
paraphrase_task = 'shakespeare'
prompt_task = paraphrase_task #+ "_prompt"


paraphrase_model_nick = "t5_transfer_shakespeare"
paraphrase_model_type = 't5-small'
output_dir = "../models/"
epochs = "3"
train_batch_size = "16"
eval_batch_size = "16"
save_log_steps = "400"

parser = HfArgumentParser((pma, pda, pta))
model_args_para, data_args_para, training_args_para = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    paraphrase_model_name,
    "--model_nick",
    paraphrase_model_nick,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, paraphrase_model_nick),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    save_log_steps,
    "--save_steps",
    save_log_steps,
    "--data_parallel",
    "True"
])


PyTorch: setting up devices


In [15]:
joint_task = "abstract+shakespeare" #"formality+emo"
data_dir = "../data/processed_filtered/"
joint_model_name = "distilbert-base-uncased"
joint_model_nick = "distilbert_uncased_2"
output_dir = "../models/"
freeze_encoder = "False"
skip_preclassifier = "False"
train_jointly = "True"
epochs = "5"
train_batch_size = "256"
eval_batch_size = "512"
log_save_steps = "200"

parser = HfArgumentParser((jma, jda, jta))
model_args_joint, data_args_joint, training_args_joint = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    joint_model_name,
    "--model_nick",
    joint_model_nick,
    "--task",
    joint_task,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, joint_model_nick, joint_task, 'joint'),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--freeze_encoder",
    freeze_encoder,
    "--skip_preclassifier",
    skip_preclassifier,
    "--train_jointly",
    train_jointly,
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    log_save_steps,
    "--save_steps",
    log_save_steps
])


PyTorch: setting up devices


In [21]:
# Create the paraphraser tokenizer and dataset objects
para_tokenizer = AutoTokenizer.from_pretrained(paraphrase_model_type, cache_dir=model_args_para.cache_dir,
                                         model_max_length = data_args_para.max_seq_len)
dataset = ldp(data_dir, para_tokenizer, mode=mode, tasks=[prompt_task], n_proc=6000)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at ../models/cache/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
  

                                                   0    1  \
0                      I'm sure you won't marry her.  low   
1                      stand in front of the hearse!  low   
2  I'm not going to walk out of the door, but som...  low   
3                      how do you mean removing him?  mid   
4                                 I'm yours forever.  low   

                                                   2     3  
0         But thus, I trust, you will not marry her.   mid  
1                             Stand from the hearse.   mid  
2  I have no will to wander forth of doors, Yet s...  high  
3                  How do you mean, removing of him?  high  
4                            I am your own for ever.  high  


100%|██████████| 2/2 [00:00<00:00,  4.23it/s]


In [22]:
# Use the paraphrase configuration defined above to create the model
model = AutoModelWithLMHead.from_pretrained(os.path.join(output_dir, paraphrase_model_name))
#training_args_para.output_dir)

/home/dmac/Documents/MIMS Coursework/Capstone/env_marvin/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:966: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file ../models/t5_transfer_shakespeare/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0

## Use the Paraphraser to Generate Predictions

In [23]:
sampler = SequentialSampler(dataset)
dataloader = DataLoader(dataset, sampler=sampler, batch_size=16)

num_return_sequences = 3

device = ("cuda" if cuda.is_available() else "cpu") #and not self.args.no_cuda
model = model.to(device)
model.eval()
predicted1 = []
predicted2 = []
predicted3 = []

epoch_iterator = tqdm(dataloader, desc="Iteration")
with no_grad():
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)  # GPU or CPU
        generated_outputs = model.generate(input_ids= batch[0], 
                                           attention_mask = batch[1], 
                                           num_return_sequences = num_return_sequences, 
                                           num_beams = 12)
        paras = para_tokenizer.batch_decode(generated_outputs.detach().cpu().numpy(), 
                                                 skip_special_tokens=True)
        predicted1 += paras[0::3]
        predicted2 += paras[1::3]
        predicted3 += paras[2::3]

## Save results to a csv file

In [30]:
import pandas as pd

In [40]:
# Store outputs to disk using in_filename as the original texts 
# and writing outputs to out_filename

# If you want to do other parts of the dataset other than train, 
# set the mode in 'dataset' above to the desired mode and then rerun the paraphrase
# and change these filenames to point to the slice of the data you want to use (dev, test, etc.)

in_filename = f'{mode}.csv'
out_filename = f'{mode}_transfered.csv'


df_para = pd.DataFrame(data={'transfered1' : predicted1, 
                             'transfered2' : predicted2, 
                             'transfered3' : predicted3}) 

df = pd.read_csv(os.path.join(data_dir, paraphrase_task, in_filename), names =['paraphrase', 
                                                                               'para_bucket',
                                                                               'orig_text', 
                                                                               'oring_bucket'])
df['transfered1'] = df_para['transfered1']
df['transfered2'] = df_para['transfered2']
df['transfered3'] = df_para['transfered3']
df.to_csv(os.path.join(data_dir, prompt_task, out_filename), 
               header=False, index=False)

In [41]:
# Inspect some results
df.head(100)

,paraphrase,para_bucket,orig_text,oring_bucket,transfered1,transfered2,transfered3
0,I'm sure you won't marry her.,low,"But thus, I trust, you will not marry her.",mid,I am sure you will not marry her.,I know you will not marry her.,I am sure you’ll not marry her.
1,stand in front of the hearse!,low,Stand from the hearse.,mid,Stand in front of the hearse!,Stand in front of hearse!,Stand before the hearse!
2,"I'm not going to walk out of the door, but som...",low,"I have no will to wander forth of doors, Yet s...",high,"I’ll not walk out of the door, But something l...","I’ll not walk out of the door, but something l...","I will not walk out of the door, But something..."
3,how do you mean removing him?,mid,"How do you mean, removing of him?",high,How mean you to remove him?,How dost thou mean removing him?,How mean you removing him?
4,I'm yours forever.,low,I am your own for ever.,high,I am yours forever.,I am thyself forever.,I am thy lord forever.
...,...,...,...,...,...,...,...
95,"the capon burns, the pig falls out of the spit...",mid,The capon burns; the pig falls from the spit; ...,high,"The capon burns, the pig falls out of the spit...","The capon burns, the pig falls out of the spit...","The capon burns, the pig falls out of spit, Th..."
96,"I'm going to hie, and I'll give you the papers...",low,"Well, I will hie, And so bestow these papers a...",high,"I will hie, and give thee papers as much as th...","I’ll hie, and give thee papers as much as thou w","I will hie, and give thee papers As much as th..."
97,"I'm my lord, are you?",low,"Do you know me, my lord?",mid,I am my lord?,I am my lord.,Am I my lord?
98,he's a courtier.,low,"He hath been a courtier, he swears.",high,He is a courtier.,He’s a courtier.,He is courtier.


In [42]:
df.tail()

,paraphrase,para_bucket,orig_text,oring_bucket,transfered1,transfered2,transfered3
7758,"I've got a couple of messengers, madam.",low,"Ay, madam, twenty several messengers.",high,"I have, madam, a couple of messengers.","I have several messengers, madam.","I have some messengers, madam."
7759,I've never heard of it.,low,He never did harm that I heard of.,mid,I never heard of it.,I have never heard of it.,I never heard it.
7760,he's not going to be admirable.,low,"Oh, ’twill be admirable!",high,He will not be admirable.,He shall not be admirable.,He is not admirable.
7761,he hasn't claimed or deserved it.,low,This prince hath neither claimed it nor deserv...,high,He hath not claimed or deserved it.,He hath not claimed or deserved.,He hath not claimed nor deserved it.
7762,"he's going to talk to you, are you?",low,"He’ll speak with you, will you or no.",mid,Will he speak with you?,Is he going to speak with you?,Shall he speak with you?


## Now use classifier for Scoring
This may cause GPU memory issues, so it's possible you may have to shutdown the kernel and restart without running the paraphraser first to run this next portion. If doing so, reload the df that was written to disk in several cells above.  

## Load in desired dataset and classifier model
In the cell below, define the dataset you want to work with and the classifier model.

In [43]:
model_config = AutoConfig.from_pretrained(model_args_joint.model_name_or_path, 
                                          cache_dir=model_args_joint.cache_dir)
tokenizer = AutoTokenizer.from_pretrained(model_args_joint.model_name_or_path, 
                                          cache_dir=model_args_joint.cache_dir,
                                          model_max_length = data_args_joint.max_seq_len)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at ../models/cache/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.3.3",
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at ../models/cache/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd

In [44]:
# Load data as expected by joint classifier
tasks = data_args_joint.task.split('+')
train_dataset, idx_to_classes = jld(data_args_joint.data_dir, 
                                             tokenizer, 
                                             model_name=model_args_joint.model_name_or_path, 
                           tasks=tasks, mode="train", n_proc=6000)
dev_dataset, _ = jld(data_args_joint.data_dir, 
                              tokenizer, 
                              model_name=model_args_joint.model_name_or_path, 
                              tasks=tasks, mode="dev", n_proc=6000)

  0%|          | 0/6 [00:00<?, ?it/s]

torch.Size([81523, 64]) torch.Size([81523, 64]) torch.Size([81523, 2]) torch.Size([81523])


 25%|██▌       | 1/4 [00:00<00:00,  8.65it/s]

torch.Size([113262, 64]) torch.Size([113262, 64]) torch.Size([113262, 2]) torch.Size([113262])


  0%|          | 0/2 [00:00<?, ?it/s]

torch.Size([20306, 64]) torch.Size([20306, 64]) torch.Size([20306, 2]) torch.Size([20306])


100%|██████████| 2/2 [00:00<00:00,  7.74it/s]

torch.Size([28322, 64]) torch.Size([28322, 64]) torch.Size([28322, 2]) torch.Size([28322])


In [45]:
label_dims = {task : 1 if len(list(idx_to_classes[task].keys())) == 2 else len(list(idx_to_classes[task].keys())) for task in idx_to_classes}
label_dims

{'abstract': 1, 'shakespeare': 1}

In [48]:
joint_model = JointSeqClassifier.from_pretrained(os.path.join(output_dir,
                                                              model_args_joint.model_nick, joint_task),
                                           tasks=tasks,
                                           model_args=model_args_joint,
                                           task_if_single=None, 
                                           joint = training_args_joint.train_jointly,
                                           label_dims=label_dims)

trainer = JointTrainer([training_args_joint,model_args_joint, data_args_joint], 
                       joint_model, train_dataset, dev_dataset, idx_to_classes)

loading configuration file ../models/distilbert_uncased_2/abstract+shakespeare/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "JointSeqClassifier"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.3.3",
  "vocab_size": 30522
}

loading weights file ../models/distilbert_uncased_2/abstract+shakespeare/pytorch_model.bin
All model checkpoint weights were used when initializing JointSeqClassifier.

All the weights of JointSeqClassifier were initialized from the model checkpoint at ../models/distilbert_uncased_2/abstract+shakespeare.
If your task is similar to the task the model of the

## Run classifier on paraphrased and original text

This is currently done with pd DataFrames but could probably be made better by using a batch data loader. 

In [49]:
import scipy.stats as ss
from tqdm import tqdm
tqdm.pandas()

In [50]:
tasks

['abstract', 'shakespeare']

In [51]:
def pred_paraphrases(row, tasks, cols):
    '''
    Make style predictions on a given df row for a given set of text columns
    and classification tasks. 
    '''
    preds = {}
    for col in cols:
        sentence = row[col]
        out = trainer.predict_for_sentence(sentence, tokenizer)
        for task in tasks:
            pred = float(out[task]['prob'])
            preds[task + '_' + col] = pred
    return preds

def get_best_pred(row, cols, target_val=0.5):
    '''
    Helper funtion for determiningg which paraphrase is 'best' 
    for a given set of paraphrase column style scores and a target value
    that you want the scores to be close to. Currently just outputs the best score
    but could be modified to get best sentence as well.
    '''
    best_diff = 1
    best_val = None
    for col in cols:
        diff = abs(row[col] - target_val)
        if diff < best_diff:
            best_val = row[col]
            best_diff = diff
    return best_val

In [53]:
# Define columns on which to run the classification
cols_to_use = ['paraphrase','transfered1', 'transfered2', 'transfered3']
# Define the names of the columns where the output scores will be stored
cols_preds = [f'pred_{tasks[0]}_orig', f'pred_{tasks[1]}_orig',
              f'pred_{tasks[0]}_transfered1', f'pred_{tasks[1]}_transfered1',
              f'pred_{tasks[0]}_transfered2', f'pred_{tasks[1]}_transfered2',
              f'pred_{tasks[0]}_transfered3', f'pred_{tasks[1]}_transfered3']
# Store results into df
df[cols_preds] = df.progress_apply(lambda x : pred_paraphrases(x, tasks, cols_to_use), 
                                   axis=1, result_type="expand")

100%|██████████| 7763/7763 [04:05<00:00, 31.62it/s]


In [54]:
# Store results of style classification:
out_filename = paraphrase_task + f'_{mode}_cross_predict_transfers.csv'

df.to_csv(os.path.join(data_dir, prompt_task, out_filename), header=True, index=False)

## Analysing the results of the transfer for style changes

In [55]:
df.head(100)

,paraphrase,para_bucket,orig_text,oring_bucket,transfered1,transfered2,transfered3,pred_abstract_orig,pred_shakespeare_orig,pred_abstract_transfered1,pred_shakespeare_transfered1,pred_abstract_transfered2,pred_shakespeare_transfered2,pred_abstract_transfered3,pred_shakespeare_transfered3
0,I'm sure you won't marry her.,low,"But thus, I trust, you will not marry her.",mid,I am sure you will not marry her.,I know you will not marry her.,I am sure you’ll not marry her.,0.000403,0.000557,0.000232,0.002530,0.000307,0.050229,0.122297,0.998928
1,stand in front of the hearse!,low,Stand from the hearse.,mid,Stand in front of the hearse!,Stand in front of hearse!,Stand before the hearse!,0.000320,0.012139,0.000320,0.012139,0.000438,0.040431,0.062622,0.998390
2,"I'm not going to walk out of the door, but som...",low,"I have no will to wander forth of doors, Yet s...",high,"I’ll not walk out of the door, But something l...","I’ll not walk out of the door, but something l...","I will not walk out of the door, But something...",0.000409,0.000621,0.164337,0.998977,0.164337,0.998977,0.000302,0.026706
3,how do you mean removing him?,mid,"How do you mean, removing of him?",high,How mean you to remove him?,How dost thou mean removing him?,How mean you removing him?,0.000276,0.000980,0.006401,0.981065,0.204749,0.998964,0.006354,0.977889
4,I'm yours forever.,low,I am your own for ever.,high,I am yours forever.,I am thyself forever.,I am thy lord forever.,0.000332,0.000957,0.058799,0.997922,0.229949,0.999094,0.320483,0.999098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"the capon burns, the pig falls out of the spit...",mid,The capon burns; the pig falls from the spit; ...,high,"The capon burns, the pig falls out of the spit...","The capon burns, the pig falls out of the spit...","The capon burns, the pig falls out of spit, Th...",0.138432,0.998865,0.007343,0.961486,0.007343,0.961486,0.029663,0.993786
96,"I'm going to hie, and I'll give you the papers...",low,"Well, I will hie, And so bestow these papers a...",high,"I will hie, and give thee papers as much as th...","I’ll hie, and give thee papers as much as thou w","I will hie, and give thee papers As much as th...",0.000431,0.000553,0.276824,0.999133,0.307574,0.999119,0.276824,0.999133
97,"I'm my lord, are you?",low,"Do you know me, my lord?",mid,I am my lord?,I am my lord.,Am I my lord?,0.002302,0.874076,0.278475,0.999096,0.251299,0.999111,0.276162,0.999099
98,he's a courtier.,low,"He hath been a courtier, he swears.",high,He is a courtier.,He’s a courtier.,He is courtier.,0.000288,0.003029,0.165066,0.998896,0.334452,0.999019,0.202655,0.998996


In [56]:
df[f'{prompt_task}_diff1'] =  abs(df[f'pred_{prompt_task}_orig'] - df[f'pred_{prompt_task}_transfered1'])
df[f'{prompt_task}_diff2'] =  abs(df[f'pred_{prompt_task}_orig'] - df[f'pred_{prompt_task}_transfered2'])
df[f'{prompt_task}_diff3'] =  abs(df[f'pred_{prompt_task}_orig'] - df[f'pred_{prompt_task}_transfered3'])

In [57]:
df[f'{prompt_task}_diff_max'] = df.apply(lambda x : np.max([x[f'{prompt_task}_diff1'], 
                                                       x[f'{prompt_task}_diff2'], 
                                                       x[f'{prompt_task}_diff3']]), axis=1)

Let's look at best sytle difference summary stats

In [58]:
df[f'{prompt_task}_diff_max'].describe()

count    7763.000000
mean        0.759769
std         0.391638
min         0.000000
25%         0.630386
50%         0.996468
75%         0.998343
max         0.998648
Name: shakespeare_diff_max, dtype: float64

Let's disaggregate by class label

In [60]:
df.columns

Index(['paraphrase', 'para_bucket', 'orig_text', 'oring_bucket', 'transfered1',
       'transfered2', 'transfered3', 'pred_abstract_orig',
       'pred_shakespeare_orig', 'pred_abstract_transfered1',
       'pred_shakespeare_transfered1', 'pred_abstract_transfered2',
       'pred_shakespeare_transfered2', 'pred_abstract_transfered3',
       'pred_shakespeare_transfered3', 'shakespeare_diff1',
       'shakespeare_diff2', 'shakespeare_diff3', 'shakespeare_diff_max'],
      dtype='object')

In [61]:
df[df['para_bucket']=='low'][f'{prompt_task}_diff_max'].describe()

count    6324.000000
mean        0.835582
std         0.333153
min         0.000000
25%         0.957236
50%         0.997673
75%         0.998396
max         0.998648
Name: shakespeare_diff_max, dtype: float64

In [63]:
df[df['para_bucket']=='mid'][f'{prompt_task}_diff_max'].describe()

count    1439.000000
mean        0.426594
std         0.451128
min         0.000000
25%         0.001393
50%         0.132307
75%         0.986538
max         0.998554
Name: shakespeare_diff_max, dtype: float64

In [66]:
df.iloc[99]

paraphrase                              I'm sure you'll come tomorrow.
para_bucket                                                        low
orig_text                       And I beseech you come again tomorrow.
oring_bucket                                                      high
transfered1                           I am sure thou come’st tomorrow.
transfered2                       I am sure thou willst come tomorrow.
transfered3                           I know thou shalt come tomorrow.
pred_abstract_orig                                            0.000351
pred_shakespeare_orig                                         0.000657
pred_abstract_transfered1                                     0.315657
pred_shakespeare_transfered1                                  0.999105
pred_abstract_transfered2                                     0.282092
pred_shakespeare_transfered2                                  0.999115
pred_abstract_transfered3                                     0.313481
pred_s

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()